In [1]:
import torchvision
from PIL import Image
import os
import torch
from torchvision import transforms


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def read_file(filename):
    img_names = os.listdir(filename)
    img_list = [os.path.join(filename,name) for name in img_names if name.endswith('.jpg')]
    return img_list

filename = './train'
filename_t = './test'

split_train = './train_split/'
split_test = './test_split/'


#获取训练集
img_train = read_file(filename)
img_test = read_file(filename_t)
#获取类别标签
img_target = [img_path.split(os.sep)[-1].split('.')[0] for img_path in img_train]
    
def mk_dir(path):
    folder = os.path.exists(path)
    if not folder:
        os.makedirs(path)
    else:
        pass


def split_img(img_set,i,path):
    print(img_set.split(os.sep)[-1])
    img = Image.open(img_set)
    img_name = img_set.split(os.sep)[-1].split('.')[0]
    w, h = img.size 
    for c in range(5):
        box = (c * 30, 0, (c+1)* 30, 30)
        mk_dir(path)
        img.crop(box).save(path+os.sep+str(i)+img_name[c]+'.jpg')


# for i in range(len(img_train)):

#     split_img(img_train[i],i,'./train_split/')



In [2]:
from torchvision import datasets
import torch.utils.data as Data
import pandas as pd
import cv2 as cv
import numpy as np
from PIL import Image
import torch
import os
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor()
])

batch_size = 64

class Codeimg(Data.Dataset):
    
    def __init__(self, root, transform=None):
        self.root = root
        self.paths = os.listdir(root)
        self.transforms = transform   
        
    def one_hot(self, label):
        bp = torch.Tensor([])
        for i in range(len(label)):
            num = ord(label[i])-48
            if num>9:
                num -= 7
                if num>35:
                    num -= 6
            a = num
            b = torch.zeros(1, 62)
            b[:,a] = 1
            bp = torch.cat((bp,b),dim=1)
        return bp
        
    def __getitem__(self, index):
        image_path = self.paths[index]    
        label = list(image_path)[:-4]
        label = self.one_hot(label).reshape(310)

        pil_image = Image.open(self.root+image_path)
        if self.transforms:
            data = self.transforms(pil_image)
        else:
            image_array = np.asarray(pil_image)
            data = torch.from_numpy(image_array)
        return data, label

    def __len__(self):
        return len(self.paths)


dataset = Codeimg('./train/', transform)
train_loader = Data.DataLoader(dataset,batch_size = 32, shuffle=True, num_workers = 0,drop_last = False)


#testPath = read_file(split_test)
#testSet,testTarget = read_set(testPath)

In [3]:
import torch.nn as nn
import torch
import torchvision
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

In [4]:

'''
class Resnet(nn.Module):
 
    def __init__(self, ch_in, ch_out):
        super(Resnet, self).__init__()
        self.conv1 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(ch_out)
        self.conv2 = nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(ch_out)
 
        self.extra = nn.Sequential()
        # 如果输出、输出维度不同，需转化后才能相加
        if ch_out != ch_in:
            self.extra = nn.Sequential(
                nn.Conv2d(ch_in, ch_out, kernel_size=1, stride=1),
                nn.BatchNorm2d(ch_out)
            )
 
    def forward(self, x):
        out = F.relu((self.bn1(self.conv1(x))))
        out = self.bn2(self.conv2(out))
        out = self.extra(x) + out
        return out
    

class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
 
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64)
        )
        # 4 block [b, 64, h, w] => [b, 1024, h, w]
        self.blk1 = Resnet(64, 128)
        self.blk2 = Resnet(128, 256)
        #self.blk3 = Resnet(256, 512)
#         self.blk4 = Resnet(512, 1024)
        # 注意最后全连接层维度，进去之前需要先打平
        self.outlayer = nn.Linear(230400, 62)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
 
        #
        x = self.blk1(x)
        x = self.blk2(x)
 #       x = self.blk3(x)
#         x = self.blk4(x)
        x = x.view(x.size(0), -1)  # 先打平，再进全连接
 
        x = self.outlayer(x)
        return x
'''  
    

class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.fc = nn.Linear(4608, 310)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def ResNet18():
    return ResNet(ResidualBlock)

In [5]:
model = ResNet18().to(DEVICE)

eopch =  15
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

In [6]:
criterion = nn.MultiLabelSoftMarginLoss()

In [7]:
model.train()
for i in range(eopch):
    for idx, (x,label) in enumerate(train_loader):
        
        x,label = x.to(DEVICE), label.to(DEVICE)
        logists = model(x)
        loss = criterion(logists, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(i,'/',eopch, loss.item())
PATH = './model/model.pkl'
torch.save(model.state_dict(), PATH)

0 / 15 0.08207709342241287
1 / 15 0.0816788524389267
2 / 15 0.08187469094991684
3 / 15 0.08205973356962204
4 / 15 0.07850702106952667
5 / 15 0.054713498800992966
6 / 15 0.025095952674746513
7 / 15 0.013681856915354729
8 / 15 0.008831674233078957
9 / 15 0.008224298246204853
10 / 15 0.004405875690281391
11 / 15 0.004573537968099117
12 / 15 0.004606660921126604
13 / 15 0.003127943491563201
14 / 15 0.001488454407081008


In [8]:
PATH = './model/model.pkl'
c = torch.load(PATH)
# 读取 Checkpoint 中的网络参数
model.load_state_dict(c)

<All keys matched successfully>

In [9]:

class UnCodeimg(Data.Dataset):
    
    def __init__(self, root, transform=None):
        self.root = root
        self.paths = os.listdir(root)
#        self.paths = self.paths.sort(key=lambda x:int(x.split('.')[0]))
        self.transforms = transform   
        
        
    def __getitem__(self, index):
        image_path = self.paths[index]
        label = "".join(image_path[:-4])
        label = int(label)
        pil_image = Image.open(self.root+image_path)
        if self.transforms:
            data = self.transforms(pil_image)
        else:
            image_array = np.asarray(pil_image)
            data = torch.from_numpy(image_array)
        return data,label

    def __len__(self):
        return len(self.paths)

def uncode(code):
    biao = list()
    for i in code:
        i = int(i)
        if i<10:
            biao.append(chr(i+48))
        elif 10<=i<36:
            biao.append(chr(i+55))
        else: 
            biao.append(chr(i+61))
    return "".join(biao)

testSet = UnCodeimg('./test/',transform)
img_dict = dict()
for i in range(len(testSet)):
    
    result = model(testSet[i][0].view(1,3,30,150).to(DEVICE))
    result = result.view(5,62)
    result = nn.functional.softmax(result, dim = 1)
    result = torch.argmax(result, dim = 1)
    img_dict[testSet[i][1]] = uncode(result)
img_dict

# df_p = pd.DataFrame({'id':np.array(range(len(img_list)+1))[1:],'y':img_list})
# df_p.to_csv("./sample_test_target.csv",index=False)   

{0: 'hmDV6',
 1: 'fl401',
 10: '5Qd2k',
 100: 'EL0q2',
 1000: 'MrVHb',
 10000: 'mK4X0',
 10001: '4u80D',
 10002: 'IeKxh',
 10003: 'ojV8g',
 10004: '27W1b',
 10005: 'vbm87',
 10006: 'i48K7',
 10007: 'c1D8i',
 10008: 'RtPjK',
 10009: 'l8oyE',
 1001: 'EA30j',
 10010: 'pk9u8',
 10011: 'dTXzJ',
 10012: '10K4F',
 10013: '18wi8',
 10014: '5Fn1V',
 10015: 'Xus39',
 10016: 'imXg3',
 10017: '231PU',
 10018: '1P8Ji',
 10019: 's57KZ',
 1002: '3Up9V',
 10020: 'H9M7F',
 10021: 'wZ1Th',
 10022: 'Z4qBj',
 10023: 'IYu61',
 10024: 'sNzfO',
 10025: 'M0zJm',
 10026: '420zd',
 10027: '3bj5x',
 10028: 'l5gdS',
 10029: 'n5C3D',
 1003: 'UZ9h7',
 10030: 'a07Lf',
 10031: 'PVI44',
 10032: 'i4ge0',
 10033: 'lYsL3',
 10034: 'ac042',
 10035: '9uHr6',
 10036: 'X5pRn',
 10037: 'pCV0d',
 10038: '502nl',
 10039: '049S6',
 1004: '4UG6n',
 10040: '1MT0P',
 10041: '7yZ30',
 10042: 'D0aPS',
 10043: '314pi',
 10044: 'BSZVM',
 10045: '8DWq2',
 10046: 'mC7JH',
 10047: 'ylxAa',
 10048: 'EF1rP',
 10049: 'Q2xlD',
 1005: '1vMYE',

In [10]:
num = list()
lab = list()
for i in range(len(img_dict)):
    num.append(i)
    lab.append(img_dict[i])

In [11]:
df_p = pd.DataFrame({'id':np.array(range(len(lab))),'y':lab})
df_p.to_csv("./sample_test_target.csv",index=False)